In [ ]:
print("Test Scucessfully!")

Test Scucessfully!


# Load Dataset and Model

In [ ]:
from datasets import load_dataset

# Load Pre-processed Dataset from HuggingFace
dataset_name = "OneFly7/llama2-sst2-fine-tuning-without-system-info"
train_dataset = load_dataset(dataset_name, split="train")

Found cached dataset parquet (/home/9130/.cache/huggingface/datasets/OneFly7___parquet/OneFly7--llama2-sst2-fine-tuning-without-system-info-ff57b5730490a513/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [ ]:
train_dataset[0]['text']

'<s>[INST] Sentence: hide new secretions from the parental units  \nSentiment: [/INST] negative </s>'

In [ ]:
sub_dataset = train_dataset.shuffle(seed=42).select(range(100))

In [ ]:
sub_dataset[0]

{'label_text': 'positive',
 'text': '<s>[INST] Sentence: klein , charming in comedies like american pie and dead-on in election ,  \nSentiment: [/INST] positive </s>'}

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

## Version 2-13b for finetuning
base_model_name = "meta-llama/Llama-2-13b-hf"
# Version 2-13b-chat for few-shot inference
# base_model_name = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True
)

base_model.config.use_cache = False

# More info: https://github.com/huggingface/transformers/pull/24906
base_model.config.pretraining_tp = 1 

/usr/local/lib/python3.10/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Load Peft and tokenizer

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

output_dir = "./results"

In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=50,
    gradient_accumulation_steps=1,
    learning_rate=2e-4,
    logging_steps=2,
    # max_steps=200,
    num_train_epochs=20,
)

max_seq_length = 128

trainer = SFTTrainer(
    model=base_model,
    train_dataset=sub_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
)

/home/9130/.local/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
2,4.580100
4,4.298200
6,3.966800
8,3.618000
10,3.289100
12,3.010400
14,2.788800
16,2.600300
18,2.460800
20,2.382400


TrainOutput(global_step=20, training_loss=3.299476218223572, metrics={'train_runtime': 451.359, 'train_samples_per_second': 2.216, 'train_steps_per_second': 0.044, 'total_flos': 2944383062016000.0, 'train_loss': 3.299476218223572, 'epoch': 10.0})

In [ ]:
trainer.save_model("./models/llama-2-13b-hf-SFT-100-20ep")

: 

: 